# E 题 AI 辅助智能体测

对比赛提供的附件一数据进行预览、清洗等操作

# 问题1
起跳/落地时刻检测 + 滞空阶段运动描述

核心思路：
1. 数据预处理：处理异常值（如跟踪丢失导致的Y=0）
2. 起跳检测：基于脚部垂直速度由0变负（向上运动）
3. 落地检测：基于脚部垂直速度由负变0（停止运动）
4. 滞空分析：质心轨迹、关节角度、手臂摆动

---

## 基础配置

---

### 输入输出设置

输入文件夹：../data/attachments/attachment1/

输出文件夹：./output/q1

---

### 关键点索引

关键点索引（根据附件2）：
- 27/28: 左/右脚踝
- 29/30: 左/右脚跟
- 31/32: 左/右脚尖
- 25/26: 左/右膝盖
- 23/24: 左/右髋部
- 11/12: 左/右肩膀
- 13/14: 左/右肘部
- 15/16: 左/右手腕

---

### 视频帧率

由命令
```bash
ffprobe -v error -select_streams v:0 -show_entries stream=r_frame_rate -of default=noprint_wrappers=1 "Athlete_01_Video.mp4"
ffprobe -v error -select_streams v:0 -show_entries stream=r_frame_rate -of default=noprint_wrappers=1 "Athlete_02_Video.mp4"
```

均输出：`r_frame_rate=30/1`

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

OUTPUT_DIR = Path("output/q1")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
INPUT_DIR = "../data/attachments/attachment1/"

# key point
KP = {
    'nose': 0,
    'left_shoulder': 11, 'right_shoulder': 12,
    'left_elbow': 13, 'right_elbow': 14,
    'left_wrist': 15, 'right_wrist': 16,
    'left_hip': 23, 'right_hip': 24,
    'left_knee': 25, 'right_knee': 26,
    'left_ankle': 27, 'right_ankle': 28,
    'left_heel': 29, 'right_heel': 30,
    'left_foot_index': 31, 'right_foot_index': 32,
}

FPS = 30 

## 数据预处理

---

### 加载xlsx数据并转换为关键点数组

load_data(filepath)

---

### 检测有效数据范围，排除跳远后往回走的部分

find_valid_range(keypoints)

1. 计算质心X坐标（水平位移）
2. 找到X最大点（最远点，即落地位置）
3. 从最远点往后检测，当水平速度持续为负时截断
    
返回: valid_end_frame (有效数据的最后一帧索引)

---

### 预处理关键点数据

preprocess_keypoints(keypoints, invalid_threshold=100)

1. 检测异常值（如Y<100可能是跟踪丢失）
2. 插值填充异常值
3. 平滑滤波

In [28]:
def load_data(filepath):
    df = pd.read_excel(filepath)
    frames = df['帧号'].values
    
    coord_cols = [col for col in df.columns if col != '帧号']
    coords = df[coord_cols].values
    keypoints = coords.reshape(len(frames), 33, 2)
    
    print(f"Loaded: {len(frames)} frames, shape: {keypoints.shape}")
    return frames, keypoints


def find_valid_range(keypoints):
    # 质心X坐标
    com_x = (keypoints[:, KP['left_hip'], 0] + keypoints[:, KP['right_hip'], 0]) / 2
    
    # 处理异常值
    com_x = np.where(com_x > 10, com_x, np.nan)
    valid_mask = ~np.isnan(com_x)
    if np.any(~valid_mask):
        valid_idx = np.where(valid_mask)[0]
        com_x = np.interp(np.arange(len(com_x)), valid_idx, com_x[valid_idx])
    
    # 平滑
    com_x_smooth = savgol_filter(com_x, 15, 3)
    vel_x = np.gradient(com_x_smooth)
    
    # 找X最大点
    max_x_frame = np.argmax(com_x_smooth)
    
    # 从最大点往后，找到速度持续为负的起始点
    valid_end = max_x_frame
    for i in range(max_x_frame, min(max_x_frame + 30, len(vel_x) - 5)):
        # 连续5帧速度都为负，说明开始往回走
        if np.all(vel_x[i:i+5] < -0.5):
            valid_end = i
            break
    
    # 确保不超过数据长度
    valid_end = min(valid_end, len(keypoints) - 1)
    
    return valid_end


def preprocess_keypoints(keypoints, invalid_threshold=100):
    kp_clean = keypoints.copy()
    n_frames, n_kp, _ = keypoints.shape
    
    # 检测并修复异常值
    for kp_idx in range(n_kp):
        for coord in range(2):
            series = kp_clean[:, kp_idx, coord]
            
            # Y坐标异常检测（太小说明跟踪丢失）
            if coord == 1:  # Y坐标
                invalid = series < invalid_threshold
                if np.any(invalid):
                    series[invalid] = np.nan
            
            # 插值填充
            if np.any(np.isnan(series)):
                valid_idx = np.where(~np.isnan(series))[0]
                if len(valid_idx) > 0:
                    series_filled = np.interp(
                        np.arange(n_frames),
                        valid_idx,
                        series[valid_idx]
                    )
                    kp_clean[:, kp_idx, coord] = series_filled
    
    # 平滑滤波
    kp_smooth = np.zeros_like(kp_clean)
    for kp_idx in range(n_kp):
        for coord in range(2):
            kp_smooth[:, kp_idx, coord] = savgol_filter(
                kp_clean[:, kp_idx, coord], 
                window_length=11, 
                polyorder=3
            )
    
    return kp_smooth


In [35]:
frames, keypoints = load_data(INPUT_DIR + "Athlete_01_PositionData.xlsx")
print(keypoints.shape)
print(keypoints[:30:])

Loaded: 301 frames, shape: (301, 33, 2)
(301, 33, 2)
[[[308.81 198.51]
  [309.58 190.21]
  [311.42 190.31]
  ...
  [290.63 556.47]
  [331.27 551.6 ]
  [286.02 567.44]]

 [[306.87 198.18]
  [308.58 190.31]
  [310.68 190.64]
  ...
  [292.02 555.18]
  [337.14 550.98]
  [289.74 566.69]]

 [[305.95 197.57]
  [308.1  190.31]
  [310.29 190.69]
  ...
  [293.22 553.05]
  [338.58 549.91]
  [291.33 565.51]]

 ...

 [[326.07 194.26]
  [323.66 187.53]
  [324.44 187.67]
  ...
  [274.31 555.54]
  [346.31 548.  ]
  [296.33 569.4 ]]

 [[328.38 194.27]
  [325.79 187.34]
  [326.67 187.35]
  ...
  [272.96 555.62]
  [347.27 547.85]
  [296.97 567.41]]

 [[331.32 194.54]
  [328.61 187.4 ]
  [329.37 187.37]
  ...
  [271.65 555.63]
  [350.31 547.09]
  [297.2  566.29]]]


## 起跳/落地检测

---

### 获取脚部最低点的Y坐标

get_foot_lowest_y(keypoints)，
也就是三个脚部点中的最大Y

---

### 检测起跳和落地时刻

detect_jump_phases(keypoints, fps=30)

算法：
1. 计算脚部Y坐标的垂直速度
2. 起跳：速度从~0变为显著负值（Y减小=上升）
3. 落地：速度从负值恢复到~0
    
返回: (takeoff_frame, landing_frame, info_dict)

---

In [30]:
def get_foot_lowest_y(keypoints):
    ankle_y = (keypoints[:, KP['left_ankle'], 1] + keypoints[:, KP['right_ankle'], 1]) / 2
    heel_y = (keypoints[:, KP['left_heel'], 1] + keypoints[:, KP['right_heel'], 1]) / 2
    toe_y = (keypoints[:, KP['left_foot_index'], 1] + keypoints[:, KP['right_foot_index'], 1]) / 2
    
    return np.maximum(np.maximum(ankle_y, heel_y), toe_y)


def detect_jump_phases(keypoints, fps=30):

    foot_y = get_foot_lowest_y(keypoints)
    foot_smooth = savgol_filter(foot_y, 15, 3)
    
    # 计算速度
    velocity = np.gradient(foot_smooth)
    velocity_smooth = savgol_filter(velocity, 11, 3)
    
    # 找最大上升速度点（速度最负）
    min_vel_idx = np.argmin(velocity_smooth)
    min_vel = velocity_smooth[min_vel_idx]
    
    # 阈值：最大速度的20%
    threshold = min_vel * 0.2
    
    # 向前搜索起跳点
    takeoff = min_vel_idx
    for i in range(min_vel_idx, -1, -1):
        if velocity_smooth[i] > threshold:
            takeoff = i
            break
    
    # 向后搜索落地点
    landing = min_vel_idx
    for i in range(min_vel_idx, len(velocity_smooth)):
        if velocity_smooth[i] > -threshold:
            landing = i
            break
    
    # 计算基准线和跳跃高度
    baseline = np.median(foot_smooth[:min(50, takeoff)])
    min_y = np.min(foot_smooth[takeoff:landing+1])
    height_pixels = baseline - min_y
    
    info = {
        'foot_y': foot_y,
        'foot_smooth': foot_smooth,
        'velocity': velocity_smooth,
        'baseline': baseline,
        'height_pixels': height_pixels,
        'min_vel_idx': min_vel_idx,
    }
    
    return takeoff, landing, info

## 运动学分析

--- 

### 计算质心位置

calculate_center_of_mass(keypoints)

这里为了简单，使用使用髋部中点。后续可以尝试**更加精确的计算**

---

### 计算三点夹角（p2为顶点）

calculate_joint_angle(p1, p2, p3)

p1, p2, p3: shape (n, 2) 或 (2,)

---

### 分析滞空阶段的运动特征

analyze_flight_phase(keypoints, takeoff, landing, fps=30)

1. 基本信息
2. 质心轨迹
3. 最高点
4. 初始速度估算（使用前几帧）
5. 关节角度和躯干倾角
6. 手臂运动

---



In [17]:
def calculate_center_of_mass(keypoints):
    left_hip = keypoints[:, KP['left_hip']]
    right_hip = keypoints[:, KP['right_hip']]
    return (left_hip + right_hip) / 2


def calculate_joint_angle(p1, p2, p3):
    
    v1 = p1 - p2
    v2 = p3 - p2
    
    if v1.ndim == 1:
        cos_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2) + 1e-8)
    else:
        cos_angle = np.sum(v1 * v2, axis=1) / (
            np.linalg.norm(v1, axis=1) * np.linalg.norm(v2, axis=1) + 1e-8
        )
    
    cos_angle = np.clip(cos_angle, -1, 1)
    return np.degrees(np.arccos(cos_angle))


def analyze_flight_phase(keypoints, takeoff, landing, fps=30):
   
    flight_kp = keypoints[takeoff:landing+1]
    n_flight = len(flight_kp)
    
    analysis = {}
    
    # 1. 基本信息
    analysis['flight_time'] = n_flight / fps
    analysis['n_frames'] = n_flight
    
    # 2. 质心轨迹
    com = calculate_center_of_mass(flight_kp)
    analysis['com'] = {
        'x': com[:, 0],
        'y': com[:, 1],
        'x_displacement': com[-1, 0] - com[0, 0],
        'y_displacement': com[-1, 1] - com[0, 1],
    }
    
    # 3. 最高点
    peak_idx = np.argmin(com[:, 1])  # Y最小 = 最高点
    analysis['peak'] = {
        'frame': takeoff + peak_idx,
        'relative_frame': peak_idx,
        'time': peak_idx / fps,
    }
    
    # 4. 初始速度估算（使用前几帧）
    if n_flight >= 3:
        dt = 1 / fps
        v0_x = (com[2, 0] - com[0, 0]) / (2 * dt)
        v0_y = -(com[2, 1] - com[0, 1]) / (2 * dt)  # Y向上为正
        
        analysis['initial_velocity'] = {
            'vx': v0_x,
            'vy': v0_y,
            'magnitude': np.sqrt(v0_x**2 + v0_y**2),
            'angle': np.degrees(np.arctan2(v0_y, v0_x)),
        }
    
    # 5. 关节角度
    knee_left = calculate_joint_angle(
        flight_kp[:, KP['left_hip']],
        flight_kp[:, KP['left_knee']],
        flight_kp[:, KP['left_ankle']]
    )
    knee_right = calculate_joint_angle(
        flight_kp[:, KP['right_hip']],
        flight_kp[:, KP['right_knee']],
        flight_kp[:, KP['right_ankle']]
    )
    hip_left = calculate_joint_angle(
        flight_kp[:, KP['left_shoulder']],
        flight_kp[:, KP['left_hip']],
        flight_kp[:, KP['left_knee']]
    )
    hip_right = calculate_joint_angle(
        flight_kp[:, KP['right_shoulder']],
        flight_kp[:, KP['right_hip']],
        flight_kp[:, KP['right_knee']]
    )
    
    # 躯干倾角
    shoulder_mid = (flight_kp[:, KP['left_shoulder']] + flight_kp[:, KP['right_shoulder']]) / 2
    hip_mid = (flight_kp[:, KP['left_hip']] + flight_kp[:, KP['right_hip']]) / 2
    trunk_vec = shoulder_mid - hip_mid
    trunk_angle = np.degrees(np.arctan2(trunk_vec[:, 0], -trunk_vec[:, 1]))
    
    analysis['angles'] = {
        'left_knee': knee_left,
        'right_knee': knee_right,
        'left_hip': hip_left,
        'right_hip': hip_right,
        'trunk': trunk_angle,
    }
    
    # 6. 手臂运动
    wrist_left = flight_kp[:, KP['left_wrist']]
    wrist_right = flight_kp[:, KP['right_wrist']]
    shoulder_y = (flight_kp[:, KP['left_shoulder'], 1] + flight_kp[:, KP['right_shoulder'], 1]) / 2
    
    analysis['arm'] = {
        'left_height': shoulder_y - wrist_left[:, 1],  # 正值=手高于肩
        'right_height': shoulder_y - wrist_right[:, 1],
    }
    
    return analysis



## 可视化

---

### 绘制起跳/落地检测结果

plot_detection_result(frames, keypoints, takeoff, landing, info, athlete_id, save_dir)

---

### 绘制滞空阶段分析

plot_flight_analysis(keypoints, takeoff, landing, analysis, athlete_id, save_dir)

---

In [18]:
def plot_detection_result(frames, keypoints, takeoff, landing, info, athlete_id, save_dir):
    """绘制起跳/落地检测结果"""
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # 图1: 脚部Y坐标
    ax = axes[0]
    ax.plot(frames, info['foot_y'], 'b-', alpha=0.3, linewidth=1, label='Raw')
    ax.plot(frames, info['foot_smooth'], 'b-', linewidth=2, label='Smoothed')
    ax.axhline(y=info['baseline'], color='gray', linestyle='--', alpha=0.7, label='Baseline')
    ax.axvline(x=takeoff, color='green', linestyle='--', linewidth=2, label=f'Takeoff (f={takeoff})')
    ax.axvline(x=landing, color='red', linestyle='--', linewidth=2, label=f'Landing (f={landing})')
    ax.fill_between(frames, info['foot_smooth'], info['baseline'],
                    where=(frames >= takeoff) & (frames <= landing),
                    alpha=0.3, color='yellow', label='Flight Phase')
    ax.set_xlabel('Frame', fontsize=12)
    ax.set_ylabel('Foot Y Position (pixels)', fontsize=12)
    ax.set_title(f'Athlete {athlete_id}: Jump Phase Detection', fontsize=14)
    ax.legend(loc='upper right')
    ax.invert_yaxis()
    ax.grid(True, alpha=0.3)
    
    # 添加滞空信息文本
    flight_time = (landing - takeoff + 1) / FPS
    ax.text(0.02, 0.98, f'Flight time: {flight_time:.3f}s\nHeight: {info["height_pixels"]:.1f}px',
            transform=ax.transAxes, fontsize=11, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # 图2: 垂直速度
    ax = axes[1]
    ax.plot(frames, info['velocity'], 'orange', linewidth=2)
    ax.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
    ax.axvline(x=takeoff, color='green', linestyle='--', linewidth=2)
    ax.axvline(x=landing, color='red', linestyle='--', linewidth=2)
    ax.axvline(x=info['min_vel_idx'], color='purple', linestyle=':', linewidth=2, label='Peak velocity')
    ax.set_xlabel('Frame', fontsize=12)
    ax.set_ylabel('Vertical Velocity (pixels/frame)', fontsize=12)
    ax.set_title('Vertical Velocity (negative = upward)', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_dir / f'athlete{athlete_id}_detection.png', dpi=150, bbox_inches='tight')
    plt.close()
    print(f"  Saved: athlete{athlete_id}_detection.png")


def plot_flight_analysis(keypoints, takeoff, landing, analysis, athlete_id, save_dir):
    """绘制滞空阶段分析"""
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    
    relative_frames = np.arange(analysis['n_frames'])
    peak_idx = analysis['peak']['relative_frame']
    
    # 图1: 质心轨迹
    ax = axes[0, 0]
    com = analysis['com']
    ax.plot(com['x'], com['y'], 'b-', linewidth=2.5)
    ax.scatter(com['x'][0], com['y'][0], s=150, c='green', marker='o', zorder=5, label='Takeoff')
    ax.scatter(com['x'][-1], com['y'][-1], s=150, c='red', marker='o', zorder=5, label='Landing')
    ax.scatter(com['x'][peak_idx], com['y'][peak_idx], s=200, c='gold', marker='*', zorder=5, label='Peak')
    ax.set_xlabel('X Position (pixels)', fontsize=11)
    ax.set_ylabel('Y Position (pixels)', fontsize=11)
    ax.set_title('Center of Mass Trajectory', fontsize=13)
    ax.legend()
    ax.invert_yaxis()
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal', adjustable='datalim')
    
    # 图2: 膝关节角度
    ax = axes[0, 1]
    angles = analysis['angles']
    ax.plot(relative_frames, angles['left_knee'], 'b-', linewidth=2, label='Left Knee')
    ax.plot(relative_frames, angles['right_knee'], 'b--', linewidth=2, label='Right Knee')
    ax.axvline(x=peak_idx, color='gold', linestyle=':', linewidth=2, label='Peak')
    ax.set_xlabel('Frame (relative)', fontsize=11)
    ax.set_ylabel('Angle (degrees)', fontsize=11)
    ax.set_title('Knee Joint Angles', fontsize=13)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 图3: 髋关节角度 + 躯干角度
    ax = axes[1, 0]
    ax.plot(relative_frames, angles['left_hip'], 'r-', linewidth=2, label='Left Hip')
    ax.plot(relative_frames, angles['right_hip'], 'r--', linewidth=2, label='Right Hip')
    ax.plot(relative_frames, angles['trunk'], 'purple', linewidth=2, label='Trunk Angle')
    ax.axvline(x=peak_idx, color='gold', linestyle=':', linewidth=2)
    ax.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
    ax.set_xlabel('Frame (relative)', fontsize=11)
    ax.set_ylabel('Angle (degrees)', fontsize=11)
    ax.set_title('Hip & Trunk Angles', fontsize=13)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 图4: 手臂高度
    ax = axes[1, 1]
    arm = analysis['arm']
    ax.plot(relative_frames, arm['left_height'], 'g-', linewidth=2, label='Left Arm')
    ax.plot(relative_frames, arm['right_height'], 'g--', linewidth=2, label='Right Arm')
    ax.axhline(y=0, color='k', linestyle='-', linewidth=0.5, label='Shoulder level')
    ax.axvline(x=peak_idx, color='gold', linestyle=':', linewidth=2)
    ax.set_xlabel('Frame (relative)', fontsize=11)
    ax.set_ylabel('Height above shoulder (pixels)', fontsize=11)
    ax.set_title('Arm Position (positive = above shoulder)', fontsize=13)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_dir / f'athlete{athlete_id}_flight.png', dpi=150, bbox_inches='tight')
    plt.close()
    print(f"  Saved: athlete{athlete_id}_flight.png")


def plot_skeleton_sequence(keypoints, takeoff, landing, athlete_id, save_dir, n_show=6):
    """绘制滞空阶段的骨架序列"""
    indices = np.linspace(takeoff, landing, n_show, dtype=int)
    
    fig, axes = plt.subplots(1, n_show, figsize=(3*n_show, 6))
    
    # 骨架连接
    bones = [
        (11, 12), (11, 13), (13, 15), (12, 14), (14, 16),  # 上肢
        (11, 23), (12, 24), (23, 24),  # 躯干
        (23, 25), (25, 27), (27, 31),  # 左腿
        (24, 26), (26, 28), (28, 32),  # 右腿
    ]
    
    for ax, frame_idx in zip(axes, indices):
        kp = keypoints[frame_idx]
        
        for start, end in bones:
            ax.plot([kp[start, 0], kp[end, 0]], 
                   [kp[start, 1], kp[end, 1]], 
                   'b-', linewidth=2.5)
        
        # 关键点
        ax.scatter(kp[:, 0], kp[:, 1], c='red', s=25, zorder=5)
        
        ax.set_title(f'Frame {frame_idx}', fontsize=11)
        ax.invert_yaxis()
        ax.set_aspect('equal')
        ax.axis('off')
    
    plt.suptitle(f'Athlete {athlete_id}: Skeleton During Flight', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.savefig(save_dir / f'athlete{athlete_id}_skeleton.png', dpi=150, bbox_inches='tight')
    plt.close()
    print(f"  Saved: athlete{athlete_id}_skeleton.png")

## 报告生成

---

### 生成详细分析报告

generate_report(athlete_id, score, takeoff, landing, analysis, fps=30)

---

### 生成滞空阶段的文字描述

describe_flight_motion(analysis)

---

In [19]:
def generate_report(athlete_id, score, takeoff, landing, analysis, fps=30):
    """生成详细分析报告"""
    lines = []
    lines.append("=" * 60)
    lines.append(f"运动者 {athlete_id} 滞空阶段分析报告")
    lines.append(f"跳远成绩: {score}")
    lines.append("=" * 60)
    
    # 基本信息
    lines.append("\n【时间信息】")
    lines.append(f"  起跳帧: {takeoff} (t = {takeoff/fps:.3f}s)")
    lines.append(f"  落地帧: {landing} (t = {landing/fps:.3f}s)")
    lines.append(f"  滞空时间: {analysis['flight_time']:.3f}s ({analysis['n_frames']} frames)")
    lines.append(f"  最高点: 起跳后第 {analysis['peak']['relative_frame']} 帧 (t = {analysis['peak']['time']:.3f}s)")
    
    # 质心运动
    lines.append("\n【质心运动】")
    com = analysis['com']
    lines.append(f"  水平位移: {com['x_displacement']:.1f} pixels")
    lines.append(f"  垂直位移: {com['y_displacement']:.1f} pixels")
    
    if 'initial_velocity' in analysis:
        iv = analysis['initial_velocity']
        lines.append(f"  初始速度: {iv['magnitude']:.1f} pixels/s")
        lines.append(f"  起跳角度: {iv['angle']:.1f}°")
    
    # 关节角度
    lines.append("\n【关节角度变化】")
    angles = analysis['angles']
    
    knee_avg = (angles['left_knee'] + angles['right_knee']) / 2
    lines.append(f"  膝关节: {knee_avg.min():.1f}° ~ {knee_avg.max():.1f}° (范围: {knee_avg.max()-knee_avg.min():.1f}°)")
    
    hip_avg = (angles['left_hip'] + angles['right_hip']) / 2
    lines.append(f"  髋关节: {hip_avg.min():.1f}° ~ {hip_avg.max():.1f}° (范围: {hip_avg.max()-hip_avg.min():.1f}°)")
    
    trunk = angles['trunk']
    lines.append(f"  躯干倾角: {trunk.min():.1f}° ~ {trunk.max():.1f}° (变化: {trunk.max()-trunk.min():.1f}°)")
    
    # 手臂摆动
    lines.append("\n【手臂摆动】")
    arm = analysis['arm']
    left_range = arm['left_height'].max() - arm['left_height'].min()
    right_range = arm['right_height'].max() - arm['right_height'].min()
    lines.append(f"  左臂摆动幅度: {left_range:.1f} pixels")
    lines.append(f"  右臂摆动幅度: {right_range:.1f} pixels")
    
    # 运动描述
    lines.append("\n【滞空阶段运动描述】")
    lines.append(describe_flight_motion(analysis))
    
    return '\n'.join(lines)


def describe_flight_motion(analysis):
    """生成滞空阶段的文字描述"""
    desc = []
    
    # 起跳阶段
    if 'initial_velocity' in analysis:
        angle = analysis['initial_velocity']['angle']
        if angle > 0:
            desc.append(f"  1. 起跳阶段: 运动者以约 {angle:.0f}° 的角度向前上方跃起，")
        else:
            desc.append(f"  1. 起跳阶段: 运动者向前跃起，")
    
    # 上升阶段
    peak_frame = analysis['peak']['relative_frame']
    n_frames = analysis['n_frames']
    peak_ratio = peak_frame / n_frames
    
    angles = analysis['angles']
    knee_at_peak = (angles['left_knee'][peak_frame] + angles['right_knee'][peak_frame]) / 2
    
    desc.append(f"  2. 上升阶段 (0~{peak_frame}帧): 身体上升至最高点，")
    desc.append(f"     膝关节角度约 {knee_at_peak:.0f}°，")
    
    # 判断收腿动作
    knee_min = (angles['left_knee'].min() + angles['right_knee'].min()) / 2
    if knee_min < 90:
        desc.append(f"     有明显的收腿动作（膝关节最小角度 {knee_min:.0f}°）。")
    else:
        desc.append(f"     收腿幅度较小。")
    
    # 下降阶段
    desc.append(f"  3. 下降阶段 ({peak_frame}~{n_frames-1}帧): 身体下落准备落地，")
    
    knee_at_end = (angles['left_knee'][-1] + angles['right_knee'][-1]) / 2
    desc.append(f"     落地时膝关节角度约 {knee_at_end:.0f}°，")
    
    # 手臂协调
    arm = analysis['arm']
    arm_max = max(arm['left_height'].max(), arm['right_height'].max())
    if arm_max > 50:
        desc.append(f"  4. 手臂动作: 有较大幅度的摆臂协调动作。")
    else:
        desc.append(f"  4. 手臂动作: 摆臂幅度较小。")
    
    return '\n'.join(desc)

In [ ]:
# 主函数
def analyze_athlete(filepath, athlete_id, score, save_dir):
    """分析单个运动者"""
    print(f"\n{'='*60}")
    print(f"分析运动者 {athlete_id} (成绩: {score})")
    print(f"{'='*60}")
    
    # 1. 加载数据
    frames, keypoints = load_data(filepath)
    
    # 2. 预处理
    keypoints = preprocess_keypoints(keypoints)
    
    # 3. 检测有效数据范围（排除往回走的部分）
    valid_end = find_valid_range(keypoints)
    print(f"\n[有效数据范围检测]")
    print(f"  原始帧数: {len(frames)}")
    print(f"  有效范围: 0 ~ {valid_end} (共{valid_end+1}帧)")
    print(f"  排除帧数: {len(frames) - valid_end - 1} (往回走的数据)")
    
    # 裁剪数据
    frames = frames[:valid_end+1]
    keypoints = keypoints[:valid_end+1]
    
    # 4. 检测起跳/落地
    print("\n[检测起跳/落地时刻]")
    takeoff, landing, detection_info = detect_jump_phases(keypoints, fps=FPS)
    print(f"  起跳帧: {takeoff} (t={takeoff/FPS:.3f}s)")
    print(f"  落地帧: {landing} (t={landing/FPS:.3f}s)")
    print(f"  滞空时间: {(landing-takeoff+1)/FPS:.3f}s")
    print(f"  跳跃高度: {detection_info['height_pixels']:.1f} pixels")
    
    # 5. 滞空分析
    print("\n[分析滞空阶段]")
    analysis = analyze_flight_phase(keypoints, takeoff, landing, fps=FPS)
    
    # 6. 可视化
    print("\n[生成图表]")
    plot_detection_result(frames, keypoints, takeoff, landing, detection_info, athlete_id, save_dir)
    plot_flight_analysis(keypoints, takeoff, landing, analysis, athlete_id, save_dir)
    plot_skeleton_sequence(keypoints, takeoff, landing, athlete_id, save_dir)
    
    # 7. 生成报告
    report = generate_report(athlete_id, score, takeoff, landing, analysis, fps=FPS)
    print(report)
    
    with open(save_dir / f'athlete{athlete_id}_report.txt', 'w', encoding='utf-8') as f:
        f.write(report)
    print(f"\n  Saved: athlete{athlete_id}_report.txt")
    
    return {
        'athlete_id': athlete_id,
        'score': score,
        'takeoff': takeoff,
        'landing': landing,
        'valid_end': valid_end,
        'analysis': analysis,
        'keypoints': keypoints,
    }


def main():
    """主函数"""
    print("=" * 60)
    print("问题1: 起跳/落地检测 + 滞空阶段分析")
    print("=" * 60)
    
    # 数据配置
    athletes = [
        {"id": 1, "file": INPUT_DIR + "Athlete_01_PositionData.xlsx", "score": "1.58米"},
        {"id": 2, "file": INPUT_DIR + "Athlete_01_PositionData.xlsx", "score": "1.15米"},
    ]
    
    results = {}
    
    for athlete in athletes:
        filepath = Path(athlete["file"])
        if filepath.exists():
            results[athlete["id"]] = analyze_athlete(
                filepath, athlete["id"], athlete["score"], OUTPUT_DIR
            )
        else:
            print(f"Warning: {filepath} not found")
    
    # 对比总结
    if len(results) == 2:
        print("\n" + "=" * 60)
        print("两位运动者对比")
        print("=" * 60)
        
        for aid in [1, 2]:
            r = results[aid]
            a = r['analysis']
            print(f"\n运动者{aid} ({r['score']}):")
            print(f"  滞空时间: {a['flight_time']:.3f}s")
            if 'initial_velocity' in a:
                print(f"  起跳速度: {a['initial_velocity']['magnitude']:.0f} px/s")
                print(f"  起跳角度: {a['initial_velocity']['angle']:.1f}°")
            
            knee = (a['angles']['left_knee'] + a['angles']['right_knee']) / 2
            print(f"  膝关节收缩: {knee.max():.0f}° → {knee.min():.0f}°")
    
    print(f"\n所有结果已保存至: {OUTPUT_DIR.absolute()}")
    return results


if __name__ == "__main__":
    main()

问题1: 起跳/落地检测 + 滞空阶段分析

分析运动者 1 (成绩: 1.58米)
Loaded: 301 frames, shape: (301, 33, 2)

[有效数据范围检测]
  原始帧数: 301
  有效范围: 0 ~ 213 (共214帧)
  排除帧数: 87 (往回走的数据)

[检测起跳/落地时刻]
  起跳帧: 120 (t=4.000s)
  落地帧: 131 (t=4.367s)
  滞空时间: 0.400s
  跳跃高度: 55.4 pixels

[分析滞空阶段]

[生成图表]
  Saved: athlete1_detection.png
  Saved: athlete1_flight.png
  Saved: athlete1_skeleton.png
运动者 1 滞空阶段分析报告
跳远成绩: 1.58米

【时间信息】
  起跳帧: 120 (t = 4.000s)
  落地帧: 131 (t = 4.367s)
  滞空时间: 0.400s (12 frames)
  最高点: 起跳后第 11 帧 (t = 0.367s)

【质心运动】
  水平位移: 222.1 pixels
  垂直位移: -68.4 pixels
  初始速度: 371.9 pixels/s
  起跳角度: 164.3°

【关节角度变化】
  膝关节: 102.3° ~ 153.2° (范围: 50.9°)
  髋关节: 105.0° ~ 179.0° (范围: 74.0°)
  躯干倾角: 13.5° ~ 51.8° (变化: 38.3°)

【手臂摆动】
  左臂摆动幅度: 137.2 pixels
  右臂摆动幅度: 151.7 pixels

【滞空阶段运动描述】
  1. 起跳阶段: 运动者以约 164° 的角度向前上方跃起，
  2. 上升阶段 (0~11帧): 身体上升至最高点，
     膝关节角度约 110°，
     收腿幅度较小。
  3. 下降阶段 (11~11帧): 身体下落准备落地，
     落地时膝关节角度约 110°，
  4. 手臂动作: 有较大幅度的摆臂协调动作。

  Saved: athlete1_report.txt

分析运动者 2 (成绩: 1.15米)
Loaded: 301 frames,